In [6]:
import pandas as pd
import numpy as np 

from warnings import filterwarnings
filterwarnings('ignore')


In [7]:
original_data = pd.read_csv("cleaned_df_recom.csv").drop("Unnamed: 0",axis=1)

In [8]:
original_data

,probability_of_default,loss_given_default,expected_loss,lumos_score,rate_type,franchise,businesstype,business_type,empoyessexistingqty,borr_count,...,state_labor_force,state_civilian_labor_population,state_employment_pop_ratio,state_labor_force_participation_rate,Risk_Group,EXPLANATION_1,EXPLANATION_2,EXPLANATION_3,EXPLANATION_4,EXPLANATION_5
0,1.38,18.39,0.25,100,0,0,0,0,6,2,...,1489001.0,2247710.0,61.5,66.2,0,business_type,borrstate,t10y2y,loan_min_fico,loan_max_fico
1,1.83,30.18,0.55,94,1,0,0,1,5,2,...,6562825.0,10124996.0,62.6,64.8,4,loan_mean_fico,business_type,loan_max_fico,rate_type,naics_3
2,5.05,66.34,3.35,81,1,0,0,1,1,1,...,2840597.0,4237868.0,64.6,67.0,4,loan_max_fico,loan_mean_fico,business_type,rate_type,loan_min_fico
3,3.92,42.93,1.68,86,1,0,0,0,2,2,...,1892572.0,2754731.0,62.8,68.7,4,t10y2y,loan_min_fico,bankruptcy,loan_mean_fico,county_unemployment_rate
4,2.96,47.49,1.41,88,0,0,0,1,1,1,...,19327143.0,30994913.0,59.9,62.4,4,loan_max_fico,state_civilian_labor_population,loan_min_fico,loan_mean_fico,bankruptcy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.57,74.76,0.43,96,1,0,0,0,4,2,...,18444682.0,29249074.0,56.5,63.1,4,loan_mean_fico,loan_max_fico,loan_min_fico,borr_count,county_unemployment
99996,15.09,92.74,13.99,70,1,0,0,1,1,1,...,12030653.0,18163851.0,60.7,66.2,4,loan_max_fico,naics_5,t10y2y,current_inflation_rate,loan_mean_fico
99997,4.51,63.45,2.86,82,1,0,0,0,5,1,...,3470945.0,5326196.0,60.4,65.2,4,loan_max_fico,loan_mean_fico,borrstate,loan_min_fico,vix
99998,2.36,64.90,1.53,87,1,0,0,1,1,1,...,3777086.0,5679819.0,63.7,66.5,4,t10y2y,business_type,borrstate,lawsuit,county_unemployment_rate


In [9]:
exp_cols = original_data.columns[-5:]
exp_dfs = {}
for exp_col in exp_cols:
    print(exp_col)
    df = original_data[original_data.columns[:-5]]
    df[exp_col] = original_data[exp_col]
    exp_dfs[exp_col] = df
    
print("done!")

EXPLANATION_1
EXPLANATION_2
EXPLANATION_3
EXPLANATION_4
EXPLANATION_5
done!


In [10]:
exp_dfs['EXPLANATION_1'].columns

Index(['probability_of_default', 'loss_given_default', 'expected_loss',
       'lumos_score', 'rate_type', 'franchise', 'businesstype',
       'business_type', 'empoyessexistingqty', 'borr_count', 'loan_max_fico',
       'loan_min_fico', 'loan_mean_fico', 'collateral',
       'primary_collateral_type', 'collateral_amount', 'loan_size',
       'business_revenue', 'bankruptcy', 'lawsuit', 'current_inflation_rate',
       'expected_12mth_inflation_rate', 'fed_funds', 't10yie', 'vix',
       'vix_change', 'avg_hourly_earnings', 't10y2y',
       'county_unemployment_rate', 'state_unemployment_rate',
       'county_unemployment', 'state_unemployment', 'county_employment',
       'state_employment', 'county_labor_force', 'state_labor_force',
       'state_civilian_labor_population', 'state_employment_pop_ratio',
       'state_labor_force_participation_rate', 'Risk_Group', 'EXPLANATION_1'],
      dtype='object')

In [11]:
exp_dfs['EXPLANATION_1']['EXPLANATION_1'].unique()

array(['business_type', 'loan_mean_fico', 'loan_max_fico', 't10y2y',
       'rate_type', 'state_civilian_labor_population', 'naics_2',
       'franchise_name', '2017_naics_categorical', 'empoyessexistingqty',
       'borrstate', 'county_unemployment', 'loan_min_fico', 'fed_funds',
       'avg_hourly_earnings', 'naics_4', 'county_unemployment_rate',
       'borr_count', 'naics_5', 'county_employment', 'county_labor_force',
       'state_labor_force', 'expected_12mth_inflation_rate', 't10yie',
       'naics_3', 'naics_1', 'franchise', 'current_inflation_rate',
       'bankruptcy', 'state_employment', 'vix', 'state_unemployment_rate',
       'lawsuit', 'state_employment_pop_ratio', 'businesstype',
       'state_unemployment', 'state_labor_force_participation_rate',
       'vix_change', 'naics_risk_group'], dtype=object)

### Recommendation System V1

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [13]:
exp_df1 = exp_dfs['EXPLANATION_1']

In [14]:
explanation_terms = exp_df1['EXPLANATION_1'].unique()

In [15]:
my_data = exp_df1.copy(deep=True)

In [16]:
my_data.columns

Index(['probability_of_default', 'loss_given_default', 'expected_loss',
       'lumos_score', 'rate_type', 'franchise', 'businesstype',
       'business_type', 'empoyessexistingqty', 'borr_count', 'loan_max_fico',
       'loan_min_fico', 'loan_mean_fico', 'collateral',
       'primary_collateral_type', 'collateral_amount', 'loan_size',
       'business_revenue', 'bankruptcy', 'lawsuit', 'current_inflation_rate',
       'expected_12mth_inflation_rate', 'fed_funds', 't10yie', 'vix',
       'vix_change', 'avg_hourly_earnings', 't10y2y',
       'county_unemployment_rate', 'state_unemployment_rate',
       'county_unemployment', 'state_unemployment', 'county_employment',
       'state_employment', 'county_labor_force', 'state_labor_force',
       'state_civilian_labor_population', 'state_employment_pop_ratio',
       'state_labor_force_participation_rate', 'Risk_Group', 'EXPLANATION_1'],
      dtype='object')

### Model Based CF

In [17]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Load your data
# Prepare data for Surprise
data = my_data[['EXPLANATION_1', 'lumos_score']].copy()
data['user_id'] = range(len(data))  # Create artificial user IDs
data = data.rename(columns={'EXPLANATION_1': 'item_id'})



In [18]:
# Define rating scale
reader = Reader(rating_scale=(0, 1))  # Assuming lumos_score is normalized to [0, 1]

# Load data into Surprise format
surprise_data = Dataset.load_from_df(data[['user_id', 'item_id', 'lumos_score']], reader)


In [19]:
surprise_data

In [20]:
# Split data into train and test sets
trainset, testset = train_test_split(surprise_data, test_size=0.2)

In [21]:
# Choose a model (SVD is a popular choice for matrix factorization)
model = SVD()

In [22]:
# Train the model
model.fit(trainset)

In [23]:
# Function to recommend terms based on user input
def recommend_terms_with_model(testset, k=3):
    recommendations = []
    for user_id, _, _ in testset:
        item_ids = my_data['EXPLANATION_1'].unique()  # Get unique explanation terms
        predictions = []
        for item_id in item_ids:
            prediction = model.predict(user_id, item_id).est
            predictions.append((item_id, prediction))
        # Sort predictions by predicted rating and recommend top k terms
        top_recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:k]
        recommendations.append((user_id, top_recommendations))
    return recommendations

# Get recommended terms using model-based CF
recommended_terms_with_model = recommend_terms_with_model(testset)

# Print user ID and recommendations
for user_id, recommendations in recommended_terms_with_model:
    print(f"User ID: {user_id}")
    print("Recommended explanation terms with model-based CF:")
    for item_id, rating in recommendations:
        print(f"- {item_id}: {rating}")
    print()


User ID: 57400
Recommended explanation terms with model-based CF:
- business_type: 1
- loan_mean_fico: 1
- loan_max_fico: 1

User ID: 60662
Recommended explanation terms with model-based CF:
- business_type: 1
- loan_mean_fico: 1
- loan_max_fico: 1

User ID: 74573
Recommended explanation terms with model-based CF:
- business_type: 1
- loan_mean_fico: 1
- loan_max_fico: 1

User ID: 14295
Recommended explanation terms with model-based CF:
- business_type: 1
- loan_mean_fico: 1
- loan_max_fico: 1

User ID: 2751
Recommended explanation terms with model-based CF:
- business_type: 1
- loan_mean_fico: 1
- loan_max_fico: 1

User ID: 64761
Recommended explanation terms with model-based CF:
- business_type: 1
- loan_mean_fico: 1
- loan_max_fico: 1

User ID: 70986
Recommended explanation terms with model-based CF:
- business_type: 1
- loan_mean_fico: 1
- loan_max_fico: 1

User ID: 4405
Recommended explanation terms with model-based CF:
- business_type: 1
- loan_mean_fico: 1
- loan_max_fico: 1

Us

In [25]:
list(original_data['EXPLANATION_1'] +', '+original_data['EXPLANATION_2'] + ', '+original_data['EXPLANATION_3'])

['business_type, borrstate, t10y2y',
 'loan_mean_fico, business_type, loan_max_fico',
 'loan_max_fico, loan_mean_fico, business_type',
 't10y2y, loan_min_fico, bankruptcy',
 'loan_max_fico, state_civilian_labor_population, loan_min_fico',
 'loan_max_fico, county_unemployment, rate_type',
 'loan_max_fico, rate_type, loan_mean_fico',
 'rate_type, loan_mean_fico, county_unemployment',
 'loan_mean_fico, loan_max_fico, empoyessexistingqty',
 'loan_max_fico, state_civilian_labor_population, loan_mean_fico',
 'rate_type, loan_mean_fico, loan_max_fico',
 'loan_max_fico, naics_2, loan_mean_fico',
 'loan_max_fico, loan_mean_fico, business_type',
 'state_civilian_labor_population, fed_funds, business_type',
 'loan_max_fico, loan_min_fico, naics_2',
 'naics_2, loan_max_fico, naics_5',
 'loan_max_fico, loan_min_fico, rate_type',
 'business_type, loan_max_fico, rate_type',
 'franchise_name, empoyessexistingqty, business_type',
 'loan_mean_fico, business_type, loan_max_fico',
 '2017_naics_categorical

### Similarity Based CF

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors


In [27]:
exp_df1 = exp_dfs['EXPLANATION_1']
explanation_terms = exp_df1['EXPLANATION_1'].unique()

In [28]:
exp_df1.sample()

,probability_of_default,loss_given_default,expected_loss,lumos_score,rate_type,franchise,businesstype,business_type,empoyessexistingqty,borr_count,...,state_unemployment,county_employment,state_employment,county_labor_force,state_labor_force,state_civilian_labor_population,state_employment_pop_ratio,state_labor_force_participation_rate,Risk_Group,EXPLANATION_1
93358,3.1,65.46,2.03,85,1,0,0,1,2,2,...,248334.0,2487431.0,6288577.0,2581602.0,6536911.0,10133438.0,62.1,64.5,4,borr_count


In [29]:
my_data = exp_df1.copy(deep=True)

# sample to test
sample = my_data.sample(50, random_state=101)
sample_y = sample['EXPLANATION_1']
sample = sample.drop("EXPLANATION_1",axis=1)

my_data = my_data.drop(sample.index,axis=0) # removing sampled rows
my_data_exp = my_data["EXPLANATION_1"]
my_data = my_data.drop('EXPLANATION_1',axis=1)
features = my_data.columns

In [30]:
# Normalize features
scaler = MinMaxScaler()
my_data[features] = scaler.fit_transform(my_data[features])
sample[features] = scaler.fit_transform(sample[features])

In [31]:
# Fit nearest neighbors model
nn_model = NearestNeighbors(metric='cosine', algorithm='brute')
nn_model.fit(my_data[features])

NearestNeighbors(algorithm='brute', metric='cosine')

In [32]:
def recommend_terms(user_data, k=5):
    # Convert user_data from dictionary to array
    user_array = np.array([[user_data[feature] for feature in features]])
    _, indices = nn_model.kneighbors(user_array, n_neighbors=k)
    recommended_terms = []
    for idx in indices[0]:
        term = exp_df1.loc[idx, 'EXPLANATION_1']
        if term not in recommended_terms:
            recommended_terms.append(term)
    return recommended_terms

In [33]:
# Get recommended terms
for idx in range(len(sample.index)):
    sample_index = idx
    user_data = dict(sample.iloc[idx])
    print(f"ID {sample_index}")
    
    
    print("--"*25)

    recommended_terms = recommend_terms(user_data)
    print("Recommended explanation terms:", recommended_terms)
    print(f"Actual explanable terms: {sample_y.iloc[sample_index]}")

ID 0
--------------------------------------------------
Recommended explanation terms: ['borr_count', 'lawsuit', 'naics_5', 'loan_max_fico']
Actual explanable terms: loan_max_fico
ID 1
--------------------------------------------------
Recommended explanation terms: ['loan_mean_fico', 'fed_funds', 'loan_max_fico']
Actual explanable terms: borr_count
ID 2
--------------------------------------------------
Recommended explanation terms: ['borr_count', 'loan_max_fico', 't10y2y']
Actual explanable terms: t10yie
ID 3
--------------------------------------------------
Recommended explanation terms: ['rate_type', 'empoyessexistingqty', 'naics_2', 't10y2y', 'loan_max_fico']
Actual explanable terms: fed_funds
ID 4
--------------------------------------------------
Recommended explanation terms: ['loan_max_fico', 'rate_type', 'business_type', 'borrstate', 't10y2y']
Actual explanable terms: county_unemployment
ID 5
--------------------------------------------------
Recommended explanation terms: 

### Fine Tuning the NN Model

In [34]:
# Fine-tune nearest neighbors model
best_score = float('-inf')
best_params = {}

for leaf_size in [10, 20, 30, 40, 50]:
    for n_neighbors in [5, 10, 15, 20]:
        for algorithm in ['ball_tree', 'kd_tree', 'brute']:
            nn_model = NearestNeighbors(n_neighbors=n_neighbors, algorithm=algorithm, 
                                        leaf_size=leaf_size, metric='euclidean')
            nn_model.fit(my_data[features])
            
            sample_index = sample.index[0]
            user_data = dict(sample.iloc[0])

            # Get recommended terms
            recommended_terms = recommend_terms(user_data)
            print(f"Parameters: leaf_size={leaf_size}, n_neighbors={n_neighbors}, algorithm={algorithm}")
            print("Recommended explanation terms:", recommended_terms)
            print(f"Actual explanable terms: {sample_y.iloc[0]}")
            print()

Parameters: leaf_size=10, n_neighbors=5, algorithm=ball_tree
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=10, n_neighbors=5, algorithm=kd_tree
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=10, n_neighbors=5, algorithm=brute
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=10, n_neighbors=10, algorithm=ball_tree
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=10, n_neighbors=10, algorithm=kd_tree
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=10, n_neighbors=10, algorithm=brute
Recommended explan

Parameters: leaf_size=40, n_neighbors=20, algorithm=kd_tree
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=40, n_neighbors=20, algorithm=brute
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=50, n_neighbors=5, algorithm=ball_tree
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=50, n_neighbors=5, algorithm=kd_tree
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=50, n_neighbors=5, algorithm=brute
Recommended explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
Actual explanable terms: loan_max_fico

Parameters: leaf_size=50, n_neighbors=10, algorithm=ball_tree
Recommended explan

In [35]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 20],
    'leaf_size': [10, 20, 30, 40, 50],
    'algorithm': ['ball_tree', 'kd_tree', 'brute']
}

# Perform grid search
grid_search = GridSearchCV(nn_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(my_data, my_data_exp)

# Get best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'algorithm': 'ball_tree', 'leaf_size': 10, 'n_neighbors': 5}


In [124]:
grid_search.param_grid

{'n_neighbors': [5, 10, 15, 20],
 'leaf_size': [10, 20, 30, 40, 50],
 'algorithm': ['ball_tree', 'kd_tree', 'brute']}

In [40]:
# Fit nearest neighbors model with best parameters
best_nn_model = NearestNeighbors(n_neighbors=5, algorithm='ball_tree', leaf_size=10, metric='euclidean')
best_nn_model.fit(my_data[features])

NearestNeighbors(algorithm='ball_tree', leaf_size=10, metric='euclidean')

In [41]:
# Using best model
def recommend_terms_best_model(user_data, k=5):
    # Convert user_data from dictionary to array
    user_array = np.array([[user_data[feature] for feature in features]])
    _, indices = best_nn_model.kneighbors(user_array, n_neighbors=k)
    recommended_terms = []
    for idx in indices[0]:
        term = exp_df1.loc[idx, 'EXPLANATION_1']
        if term not in recommended_terms:
            recommended_terms.append(term)
    return recommended_terms

In [42]:
# taking sample from original data with all explanable features
scaler = MinMaxScaler()
new_sample = original_data.sample(50, random_state=101)
new_sample_y = new_sample.iloc[:, -5:]
new_sample = new_sample.drop(new_sample.iloc[:, -5:].columns, axis=1)
new_sample[features] = scaler.fit_transform(new_sample[features])

In [45]:
# Important Features list
important_features = set(['rate_type',
'businesstype',
'franchise',
'borr_count',
'loan_max_fico',
'loan_min_fico',
'loan_mean_fico',
'loan_size'])

### Recommendations!

In [75]:
# Get recommended terms
ground_truth = []
recommendations = []
ids = []

for idx in range(len(new_sample.index)):
    sample_index = idx
    user_data = dict(new_sample.iloc[idx])
    print()
    print(f"ID {sample_index}")
    
    
    print("--"*25)

    recommended_terms = recommend_terms_best_model(user_data)
    print("- All Recommended Explanation terms:", recommended_terms)
    print(f"- Actual explanable terms: {new_sample_y.iloc[sample_index,:].to_list()}")
    matching_items = list(set(recommended_terms).intersection(set(new_sample_y.iloc[sample_index,:].to_list())))
    print(f"- Matching Recommendations with Ground Truth: ", matching_items, f"-- Matches: {len(matching_items)}")
    print("- Important Recommendations:", list(important_features.intersection(set(recommended_terms))))
    
    
    ground_truth.append(new_sample_y.iloc[sample_index,:].to_list())
    recommendations.append(recommended_terms)
    ids.append(sample_index)


ID 0
--------------------------------------------------
- All Recommended Explanation terms: ['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit']
- Actual explanable terms: ['loan_max_fico', 'loan_mean_fico', 'loan_min_fico', 'borr_count', 'empoyessexistingqty']
- Matching Recommendations with Ground Truth:  ['borr_count', 'loan_max_fico'] -- Matches: 2
- Important Recommendations: ['borr_count', 'loan_max_fico']

ID 1
--------------------------------------------------
- All Recommended Explanation terms: ['loan_mean_fico', 'fed_funds', 'loan_max_fico', 'empoyessexistingqty']
- Actual explanable terms: ['borr_count', 'loan_mean_fico', 'loan_max_fico', 'loan_min_fico', 't10y2y']
- Matching Recommendations with Ground Truth:  ['loan_mean_fico', 'loan_max_fico'] -- Matches: 2
- Important Recommendations: ['loan_mean_fico', 'loan_max_fico']

ID 2
--------------------------------------------------
- All Recommended Explanation terms: ['borr_count', 'loan_max_fico', 't10y2y']
- Actual expl

- All Recommended Explanation terms: ['loan_max_fico', 'borr_count', 'business_type', 'borrstate']
- Actual explanable terms: ['t10y2y', 'borr_count', 'vix', 'loan_max_fico', 'naics_5']
- Matching Recommendations with Ground Truth:  ['borr_count', 'loan_max_fico'] -- Matches: 2
- Important Recommendations: ['borr_count', 'loan_max_fico']

ID 31
--------------------------------------------------
- All Recommended Explanation terms: ['state_civilian_labor_population', 'loan_max_fico', 'avg_hourly_earnings', 'loan_mean_fico']
- Actual explanable terms: ['borr_count', 'loan_mean_fico', 'business_type', 'empoyessexistingqty', 'naics_5']
- Matching Recommendations with Ground Truth:  ['loan_mean_fico'] -- Matches: 1
- Important Recommendations: ['loan_mean_fico', 'loan_max_fico']

ID 32
--------------------------------------------------
- All Recommended Explanation terms: ['county_labor_force', 'business_type', '2017_naics_categorical', 'loan_mean_fico', 'loan_max_fico']
- Actual explanable

### nDCG

In [64]:
from sklearn.metrics import ndcg_score

In [71]:
ground_truth[:2]

[['loan_max_fico',
  'loan_mean_fico',
  'loan_min_fico',
  'borr_count',
  'empoyessexistingqty'],
 ['borr_count', 'loan_mean_fico', 'loan_max_fico', 'loan_min_fico', 't10y2y']]

In [70]:
recommendations[:2]

[['borr_count', 'naics_5', 'loan_max_fico', 'lawsuit'],
 ['loan_mean_fico', 'fed_funds', 'loan_max_fico', 'empoyessexistingqty']]

In [72]:
# Compute nDCG
# Convert ground truth and recommendations to binary relevance scores
def binary_relevance(ground_truth, recommendations):
    binary_ground_truth = []
    binary_recommendations = []

    for user_recommendations in ground_truth:
        user_binary_ground_truth = [1 if item in user_recommendations else 0 for item in recommendations[0]]
        binary_ground_truth.append(user_binary_ground_truth)

    for user_recommendations in recommendations:
        user_binary_recommendations = [1 if item in user_recommendations else 0 for item in recommendations[0]]
        binary_recommendations.append(user_binary_recommendations)

    return binary_ground_truth, binary_recommendations

# Convert ground truth and recommendations to binary relevance scores
binary_ground_truth, binary_recommendations = binary_relevance(ground_truth, recommendations)

# Compute nDCG
ndcg = ndcg_score(binary_ground_truth, binary_recommendations)
print("nDCG:", ndcg)

nDCG: 0.7593705601844877


In [74]:
explanable_features = pd.read_csv("./archive/parlay_sample_explanable_features.csv")
explanable_features

,EXPLANATION_1_FEATURE_NAME,EXPLANATION_1_STRENGTH,EXPLANATION_1_ACTUAL_VALUE,EXPLANATION_1_QUALITATIVE_STRENGTH,EXPLANATION_2_FEATURE_NAME,EXPLANATION_2_STRENGTH,EXPLANATION_2_ACTUAL_VALUE,EXPLANATION_2_QUALITATIVE_STRENGTH,EXPLANATION_3_FEATURE_NAME,EXPLANATION_3_STRENGTH,...,EXPLANATION_8_ACTUAL_VALUE,EXPLANATION_8_QUALITATIVE_STRENGTH,EXPLANATION_9_FEATURE_NAME,EXPLANATION_9_STRENGTH,EXPLANATION_9_ACTUAL_VALUE,EXPLANATION_9_QUALITATIVE_STRENGTH,EXPLANATION_10_FEATURE_NAME,EXPLANATION_10_STRENGTH,EXPLANATION_10_ACTUAL_VALUE,EXPLANATION_10_QUALITATIVE_STRENGTH
0,business_type,-0.165458,'EXISTING BUSINESS',--,borrstate,0.129951,'KS',++,t10y2y,-0.124901,...,'7.1',++,state_labor_force,0.079548,'1489001.0',++,t10yie,-0.078435,'1.55',--
1,loan_mean_fico,-0.255019,'764.5',---,business_type,0.239443,'NEW BUSINESS',+++,loan_max_fico,-0.199095,...,'IL',+,borr_count,-0.047088,'2.0',-,t10yie,-0.045048,'1.61',-
2,loan_max_fico,0.264952,'689.0',+++,loan_mean_fico,0.202938,'689.0',++,business_type,0.139876,...,'4a',++,county_employment,-0.074971,'29960.0',--,t10yie,-0.066798,'1.63',--
3,t10y2y,0.180264,'2.71',++,loan_min_fico,-0.119959,'701.0',--,bankruptcy,-0.118594,...,'V',++,vix_change,0.063983,'-45.8',++,loan_max_fico,-0.056703,'732.0',--
4,loan_max_fico,0.311784,'734.0',+++,state_civilian_labor_population,-0.175416,'3.0994913E7',--,loan_min_fico,0.152704,...,'F',--,county_employment,0.102569,'1573536.0',++,empoyessexistingqty,0.097354,'1.0',++
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,loan_mean_fico,-0.387256,'804.0',--,loan_max_fico,-0.363487,'819.0',--,loan_min_fico,-0.333787,...,'N',--,business_type,-0.093073,'EXISTING BUSINESS',-,lawsuit,-0.088729,'N',-
99996,loan_max_fico,0.158760,'644.0',++,naics_5,0.151666,'54161a',++,t10y2y,0.146152,...,'9.0',++,vix,0.097909,'30.69',++,state_unemployment_rate,0.091158,'8.3',++
99997,loan_max_fico,0.262102,'689.0',+++,loan_mean_fico,0.194990,'689.0',++,borrstate,-0.171402,...,'1.68',++,borr_count,0.061872,'1.0',++,lawsuit,0.046045,'N',+
99998,t10y2y,-0.201739,'0.47',---,business_type,0.124063,'NEW BUSINESS',++,borrstate,-0.123552,...,'722.0',++,state_labor_force,-0.064545,'3777086.0',--,expected_12mth_inflation_rate,-0.057594,'2.7',--


In [108]:
x = dict(explanable_features[[f'EXPLANATION_{1}_FEATURE_NAME',f'EXPLANATION_{1}_STRENGTH']].values)
x

{'business_type': -0.206227653,
 'loan_mean_fico': -0.38725646,
 'loan_max_fico': 0.262102395,
 't10y2y': -0.201738819,
 'rate_type': 0.440777251,
 'state_civilian_labor_population': -0.201883736,
 'naics_2': -0.262560057,
 'franchise_name': 0.452936873,
 '2017_naics_categorical': -1.033619245,
 'empoyessexistingqty': -0.186968156,
 'borrstate': -0.317343446,
 'county_unemployment': -0.36207755,
 'loan_min_fico': 0.149303624,
 'fed_funds': 0.260333476,
 'avg_hourly_earnings': -0.419373238,
 'naics_4': 0.260111275,
 'county_unemployment_rate': 0.27820753,
 'borr_count': -0.479288961,
 'naics_5': -0.532298107,
 'county_employment': -0.237919011,
 'county_labor_force': 0.357003542,
 'state_labor_force': -0.189027848,
 'expected_12mth_inflation_rate': 0.26458819,
 't10yie': -0.220442366,
 'naics_3': 0.316610192,
 'naics_1': 0.217720489,
 'franchise': -0.307298702,
 'current_inflation_rate': 0.318743534,
 'bankruptcy': -0.23135145,
 'state_employment': -0.195507538,
 'vix': 0.188191103,
 's

In [102]:
# # Map the strengths of the recommendations
# mapped_recommendations = []
# mapped_ground_truths = []
# for idx, rc_list, gt_list in zip(new_sample.index, recommendations, ground_truth):
#     m_recoms = []
#     gt_recoms = []
#     x = list(explanable_features.iloc[new_sample.index[0],:].values)
#     extracted_data = {x[i]: x[i+1] for i in range(0, len(x)-1, 3)}
    
#     for rc, gt in zip(rc_list, gt_list):
#         try:
#             print(rc, extracted_data[rc])
#             print(gt, extracted_data[gt])
#         except:
            
        